In [ ]:
import numpy as np
from scipy.optimize import fmin_cg
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Read dataset + bit of preprocessing
train_images = np.fromfile("train-images.idx3-ubyte", dtype=np.ubyte)
train_images = train_images[16:]

train_labels = np.fromfile("train-labels.idx1-ubyte", dtype=np.ubyte)
train_labels = train_labels[8:]

test_images = np.fromfile("t10k-images.idx3-ubyte", dtype=np.ubyte)
test_images = test_images[16:]

test_labels = np.fromfile("t10k-labels.idx1-ubyte", dtype=np.ubyte)
test_labels = test_labels[8:]

In [ ]:
# Reshape into a matrix having dimensions (60000, 28 * 28)
train_images = np.reshape(train_images, (60000, 784))

# Reshape into a matrix having dimensions (10000, 28 * 28)
test_images = np.reshape(test_images, (10000, 784))

In [ ]:
# Initialise the network
m = len(train_images)
input_layer_size = 784
num_labels = 10
hidden_layer_size = 25
reg_lambda = 1

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def sigmoid_gradient(z):
    g = sigmoid(z)
    return g * (1 - g)

In [ ]:
def feedforward_propagation(X, theta0, theta1):
    m = len(X)

    a0 = np.hstack((np.ones((m, 1)), X)).T
    z1 = theta0 @ a0
    a1 = sigmoid(z1)
    a1 = np.vstack((np.ones((1, m)), a1))
    z2 = theta1 @ a1
    a2 = sigmoid(z2)
    return a2

In [ ]:
def cost(X, y, theta0, theta1, reg_lambda):
    m = len(X)
    h = feedforward_propagation(X, theta0, theta1)
    J = (-1 / m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    reg = (reg_lambda / (2 * m)) * \
        (np.sum(theta0[:, 2:] ** 2) + np.sum(theta1[:, 2:] ** 2))
    return J + reg

In [ ]:
def back_propagation(theta, X, y, reg_lambda, input_layer_size,
                     hidden_layer_size, num_labels, cost_or_grad):
    m = len(X)

    theta0 = np.reshape(theta[0 : hidden_layer_size * (input_layer_size + 1)],
        (hidden_layer_size, input_layer_size + 1))
    theta1 = np.reshape(theta[hidden_layer_size * (input_layer_size + 1) :],
        (num_labels, hidden_layer_size + 1))

    a0 = np.hstack((np.ones((m, 1)), X)).T
    z1 = theta0 @ a0
    a1 = sigmoid(z1)
    a1 = np.vstack((np.ones((1, m)), a1))
    z2 = theta1 @ a1
    h = sigmoid(z2)

    J = (-1 / m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    reg = (reg_lambda / (2 * m)) * \
        (np.sum(theta0[:, 2:] ** 2) + np.sum(theta1[:, 2:] ** 2))
    
    if cost_or_grad == 0:
        return J + reg

    d2 = h - y
    d1 = theta1[:, 1:].T @ d2 * sigmoid_gradient(z1)
    delta0 = d1 @ a0.T
    delta1 = d2 @ a1.T
    theta0[:, 0] = 0
    theta1[:, 0] = 0

    theta0_grad = (1 / m) * delta0 + (reg_lambda / m) * theta0
    theta1_grad = (1 / m) * delta1 + (reg_lambda / m) * theta1

    grad = np.append(theta0_grad.flatten(), theta1_grad.flatten())
    return grad

In [ ]:
def print_cost_accuracy(X_train, y_train, y_train_vector,
                        X_test, y_test, y_test_vector,
                        theta0, theta1):
    predictions = feedforward_propagation(X_train, theta0, theta1)
    predictions = np.argmax(predictions, axis=0)
    J = cost(X_train, y_train_vector, theta0, theta1, 0)
    print("Training set cost = {}".format(J))
    print("Training set accuracy = {}".format(np.mean(predictions == y_train) * 100))
    predictions = feedforward_propagation(X_test, theta0, theta1)
    predictions = np.argmax(predictions, axis=0)
    J = cost(X_test, y_test_vector, theta0, theta1, 0)
    print("Testing set cost = {}".format(J))
    print("Testing set accuracy = {}\n\n".format(np.mean(predictions == y_test) * 100))

In [ ]:
r_w = input("Read parameters from csv? (y/n): ")

if r_w == "y":
    theta0 = np.loadtxt("theta0.csv", delimiter=',')
    theta1 = np.loadtxt("theta1.csv", delimiter=',')
else:
    init_epsilon_0 = np.sqrt(6) / \
        (np.sqrt(input_layer_size) + np.sqrt(hidden_layer_size))
    init_epsilon_1 = np.sqrt(6) / \
        (np.sqrt(hidden_layer_size) + np.sqrt(num_labels))
    theta0 = np.random.random((hidden_layer_size, input_layer_size + 1)) \
        * 2 * init_epsilon_0 - init_epsilon_0
    theta1 = np.random.random((num_labels, hidden_layer_size + 1)) \
        * 2 * init_epsilon_1 - init_epsilon_1

In [ ]:
# One-hot encoding
train_labels_vector = np.zeros((num_labels, m))
for i in range(m):
    train_labels_vector[train_labels[i], i] = 1

test_labels_vector = np.zeros((num_labels, len(test_images)))
for i in range(len(test_images)):
    test_labels_vector[test_labels[i], i] = 1

In [ ]:
# Print inital cost and accuracy
print("Initial cost and accuracy:")
print_cost_accuracy(train_images, train_labels, train_labels_vector,
                        test_images, test_labels, test_labels_vector,
                        theta0, theta1)

In [ ]:
# Some useful initialisations
theta = np.append(theta0.flatten(), theta1.flatten())
my_cost = lambda theta: back_propagation(theta, train_images, train_labels_vector,
                                   reg_lambda, input_layer_size,
                                   hidden_layer_size, num_labels, 0)
my_gradient = lambda theta: back_propagation(theta, train_images, train_labels_vector,
                                       reg_lambda, input_layer_size,
                                       hidden_layer_size, num_labels, 1)
                                       
# Train the network
periods = 10
cost_history = np.zeros(periods + 1)
for i in range(periods):
    cost_history[i] = cost(train_images, train_labels_vector, theta0, theta1, reg_lambda)
    theta = fmin_cg(f=my_cost, x0=theta, fprime=my_gradient, maxiter=50/periods)
    
    theta0 = np.reshape(theta[0 : hidden_layer_size * (input_layer_size + 1)],
        (hidden_layer_size, input_layer_size + 1))
    theta1 = np.reshape(theta[hidden_layer_size * (input_layer_size + 1) :],
        (num_labels, hidden_layer_size + 1))
    print("\n\nPeriod {}.".format(i + 1))
    print_cost_accuracy(train_images, train_labels, train_labels_vector,
                        test_images, test_labels, test_labels_vector,
                        theta0, theta1)

In [ ]:
cost_history[periods] = cost(train_images, train_labels_vector, theta0, theta1, reg_lambda)

plt.title("Cost vs Periods")
plt.xlabel("Number of periods")
plt.ylabel("Cost")
plt.plot(cost_history)
plt.show()

In [ ]:
print("Final cost and accuracy:")
print_cost_accuracy(train_images, train_labels, train_labels_vector,
                        test_images, test_labels, test_labels_vector,
                        theta0, theta1)